In [ ]:
!/apps/share64/debian10/anaconda/anaconda-7/envs/AIIDA/bin/verdi presto --profile-name fair-workflows

In [ ]:
!verdi status # This points to system, one would either have to define an alias or add it to PATH

In [ ]:
%load_ext aiida
%aiida

In [ ]:
%verdi status

In [ ]:
%verdi storage info

In [ ]:
!/apps/share64/debian10/anaconda/anaconda-7/envs/AIIDA/bin/aiida-pseudo install sssp -x PBE -p efficiency
!/apps/share64/debian10/anaconda/anaconda-7/envs/AIIDA/bin/aiida-pseudo install sssp -x PBEsol -p efficiency

In [ ]:
%verdi storage info

In [ ]:
from aiida import engine, orm
from aiida_quantumespresso.calculations.pw import PwCalculation
from ase.build import bulk

In [ ]:
# %verdi code delete 'qe-7.1-pw' --force

In [ ]:
qe_code = orm.InstalledCode(
    append_text='',
    computer=orm.load_computer('localhost'),
    default_calc_job_plugin='quantumespresso.pw',
    description='',
    filepath_executable='/apps/share64/debian10/espresso/espresso-7.1/bin/pw.x',
    label='qe-7.1-pw',
    prepend_text='export OMP_NUM_THREADS=1',
    use_double_quotes=False,
    with_mpi= False
).store()

In [ ]:
%verdi code list
%verdi code test 'qe-7.1-pw'

In [ ]:
scf_inputs = {
    'CONTROL': {
        'calculation': 'scf',
        # 'pseudo_dir': Path('files').absolute().as_posix(),
    },
    'SYSTEM': {
        'occupations': 'smearing',
        'smearing': 'cold',
        'degauss': 0.02
    }
}

resources = {
    'num_machines': 1,
    'num_mpiprocs_per_machine': 1,
}

@engine.calcfunction
def rescale_list(structure: orm.StructureData, factor_list: orm.List):

    scaled_structure_dict = {}

    for index, scaling_factor in enumerate(factor_list.get_list()):

        ase_structure = structure.get_ase()

        new_cell = ase_structure.get_cell() * scaling_factor
        ase_structure.set_cell(new_cell, scale_atoms=True)

        scaled_structure_dict[f'structure_{index}'] = orm.StructureData(ase=ase_structure)

    return scaled_structure_dict

@engine.calcfunction
def create_eos_dictionary(**kwargs) -> orm.Dict:
    eos = {
        label: (result['volume'], result['energy'])
        for label, result in kwargs.items()
    }
    return orm.Dict(eos)

class EquationOfStateWorkChain(engine.WorkChain):
    """WorkChain to compute Equation of State using Quantum ESPRESSO."""

    @classmethod
    def define(cls, spec):
        """Specify inputs and outputs."""
        super().define(spec)
        spec.input("code", valid_type=orm.Code)
        spec.input("structure", valid_type=orm.StructureData)
        spec.input("scale_factors", valid_type=orm.List)

        spec.outline(
            cls.run_eos,
            cls.results,
        )
        spec.output("eos_dict", valid_type=orm.Dict)

    def run_eos(self):

        calcjob_dict = {}

        for label, rescaled_structure in rescale_list(self.inputs.structure, self.inputs.scale_factors).items():

            builder = PwCalculation.get_builder()
            builder.code = self.inputs.code
            builder.structure = rescaled_structure
            builder.parameters = orm.Dict(scf_inputs)
            pseudo_family = orm.load_group('SSSP/1.3/PBEsol/efficiency')
            builder.pseudos = pseudo_family.get_pseudos(structure=rescaled_structure)
            kpoints = orm.KpointsData()
            kpoints.set_kpoints_mesh([2, 2, 2])
            builder.kpoints = kpoints
            builder.metadata.options.resources = resources

            calcjob_dict[label] = self.submit(builder)

        self.ctx.labels = list(calcjob_dict.keys())

        return calcjob_dict

    def results(self):

        self.report(self.ctx)

        # label: self.ctx[label].outputs['properties'] for label in self.ctx.labels

        eos_results = {}
        for label in self.ctx.labels:
            energy = self.ctx[label].outputs.output_parameters.get_dict()['energy']
            volume = self.ctx[label].outputs.output_parameters.get_dict()['volume']
            eos_results[label] = orm.Dict({'energy': orm.Float(energy), 'volume': orm.Float(volume)})
        # }
        eos_dict = create_eos_dictionary(**eos_results)
        self.out('eos_dict', eos_dict)


In [ ]:
structure = orm.StructureData(ase=bulk('Al', a=4.05, cubic=True))

results = engine.run(
    EquationOfStateWorkChain,
    code=orm.load_code("qe-7.1-pw@localhost"),
    structure=structure,
    scale_factors=orm.List([0.9, 0.95, 1.0, 1.05, 1.1]),
)

In [ ]:
%verdi process list -ap 1

In [ ]:
%verdi process show 253

In [ ]:
%verdi process report 253

In [ ]:
%verdi calcjob outputcat 253

In [ ]:
calcjob = orm.load_node(253)
retrieved = calcjob.outputs.retrieved
path = calcjob.get_option('input_filename')
with retrieved.base.repository.open(path, mode='rb') as fhandle:
    file_contents = fhandle.read()
print(file_contents.decode('utf-8'))

In [ ]:
from matplotlib import pyplot as plt

eos_dict = results['eos_dict'].get_dict()

plt.plot(
    [e[0] for e in eos_dict.values()],
    [v[1] for v in eos_dict.values()],
)